In [157]:
import os
import pickle
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm

import cv2

In [158]:
test_subset_id = 9

In [159]:
download_folder = "../downloads"
download_data_folder = "../downloads/data"
download_mask_folder = "../downloads/seg-lungs-LUNA16"

output_nodules_folder = "output/nodules"

data_image_folder = "../data/images"

gt_cache_path = "../data/cache/train_gt"
no_gt_cache_path = "../data/cache/train_no_gt"
test_cache_path = "../data/cache/test"


In [160]:
for p in [output_nodules_folder, data_image_folder]:
    if not os.path.exists(p):
        os.makedirs(p)

In [161]:
image_data_gt, image_data_no_gt, image_data_test, all_image_data = [], [], [],[]

In [162]:
image_size = (512, 512)

In [163]:
annotations_path = "../downloads/annotations.csv"
annotations_df = pd.read_csv(annotations_path, delimiter=",")

In [164]:
px = 1 / plt.rcParams['figure.dpi']
plt.figure(figsize=(image_size[0] * px, image_size[1] * px))
plt.subplots_adjust(0, 0, 1, 1, 0, 0)

def show_image(image):
    plt.imshow(image)
    plt.gray()
    plt.axis('off')
    plt.show()

<Figure size 512x512 with 0 Axes>

In [165]:
def normalize(image):
    image[image > 400] = 400
    image[image < -1000] = -1000

    max_hu = image.max()
    min_hu = image.min()
    image = (image - min_hu) / (max_hu - min_hu) * 255
    image = np.round(image)
    return image

In [166]:
def world_to_voxel_coord(world_coord, origin, spacing):
    stretched_voxel_coord = np.absolute(world_coord - origin)
    voxel_coord = stretched_voxel_coord / spacing
    return voxel_coord

In [167]:
def extract_annotations(series_id, origins, spacings, is_flip):
    annotations = annotations_df[annotations_df["seriesuid"] == series_id]
    if annotations.size == 0:
        return pd.DataFrame([], columns=["z", "bboxes", "filepath", "ignoreareas"])

    nodules_annotation = []

    for index, row in annotations.iterrows():
        world_x = row["coordX"]
        world_y = row["coordY"]
        world_z = row["coordZ"]
        diameter = row["diameter_mm"]

        world_coord = np.array([float(world_z), float(world_y), float(world_x)])
        voxel_coord = world_to_voxel_coord(world_coord, origins, spacings)

        voxel_z, voxel_y, voxel_x = voxel_coord
        if is_flip:
            voxel_x = image_size[0] - voxel_x
            voxel_y = image_size[1] - voxel_y

        radius = diameter / spacings[2] / 2

        annotation = {"z": round(voxel_z), 'bboxes': np.array([
            np.array([
                max(round(voxel_x - radius), 0),
                max(round(voxel_y - radius), 0),
                min(round(voxel_x + radius), image_size[0] - 1),
                min(round(voxel_y + radius), image_size[1] - 1)
            ])
        ]), 'ignoreareas': np.array([])}
        nodules_annotation.append(annotation)

    nodules_df = pd.DataFrame(nodules_annotation)

    return nodules_df

In [168]:
def process_image(series_id, subset_id, generate_picture=True):
    mhd_file_path = "{}/subset{}/{}.mhd".format(download_data_folder, subset_id, series_id)
    mask_file_path = "{}/{}.mhd".format(download_mask_folder, series_id)

    itk_image = sitk.ReadImage(mhd_file_path, sitk.sitkFloat32)
    ct_scans = sitk.GetArrayFromImage(itk_image)

    origins = np.array(list(reversed(itk_image.GetOrigin())))
    spacings = np.array(list(reversed(itk_image.GetSpacing())))

    masks = sitk.GetArrayFromImage(sitk.ReadImage(mask_file_path, sitk.sitkFloat32))

    direction = itk_image.GetDirection()
    direction = np.array(list(map(lambda x: round(x), direction)))

    if np.any(direction != np.array([1, 0, 0, 0, 1, 0, 0, 0, 1])):
        is_flip = True
    else:
        is_flip = False

    if is_flip:
        ct_scans = ct_scans[:, ::-1, ::-1]
        masks = masks[:, ::-1, ::-1]

    ct_scans = normalize(ct_scans)

    ct_scans = ct_scans.astype(np.uint8)
    masks = masks.astype(np.uint8)

    nodules_df = extract_annotations(series_id, origins, spacings, is_flip)

    for i in range(0, ct_scans.shape[0]):
        image_filename = "{}-ss{}-z{}.jpg".format(series_id, subset_id, i)
        filepath = "data/images/{}".format(image_filename)

        nodule_row = nodules_df[nodules_df["z"] == i]

        if nodule_row.size != 0:
            nodule = nodule_row.iloc[0]
            bbox = nodule["bboxes"][0]
            bbox_x1, bbox_y1, bbox_x2, bbox_y2 = bbox

        if generate_picture:
            masked_image = cv2.bitwise_and(ct_scans[i], ct_scans[i], mask=masks[i])
            colored_image = cv2.cvtColor(masked_image,cv2.COLOR_GRAY2RGB)
            cv2.imwrite("{}/{}".format(data_image_folder, image_filename), colored_image)

            if nodule_row.size != 0:
                patched_image = cv2.rectangle(colored_image, (bbox_x1, bbox_y1), (bbox_x2, bbox_y2), (0, 0, 255), 1)
                cv2.imwrite("{}/subset{}-{}-z{}.jpg".format(output_nodules_folder, subset_id, series_id , i), patched_image)

        if subset_id != test_subset_id:
            if nodule_row.size == 0:
                anno = {'bboxes': [], 'ignoreareas': [],
                        'filepath': filepath}
                image_data_no_gt.append(anno)
                all_image_data.append(anno)
            else:
                nodule = nodule_row.iloc[0]
                anno = {'bboxes': nodule["bboxes"], 'ignoreareas': nodule["ignoreareas"],
                        'filepath': filepath}
                image_data_gt.append(anno)
                all_image_data.append(anno)
        else:
            if nodule_row.size != 0:
                nodule = nodule_row.iloc[0]
                anno = {'bboxes': nodule["bboxes"], 'ignoreareas': nodule["ignoreareas"],
                        'filepath':filepath}
                image_data_test.append(anno)
                all_image_data.append(anno)


In [169]:
for subset_id in tqdm(range(0, 10)):
    current_folder = os.listdir("{}/subset{}".format(download_data_folder, subset_id))

    for file in current_folder:
        if file.endswith(".mhd"):
            series_id = re.search(r'(.*)\.mhd', file).group(1)
            process_image(series_id, subset_id, generate_picture=False)

100%|██████████| 10/10 [12:41<00:00, 76.10s/it]


In [170]:
image_data_gt_df = pd.DataFrame(image_data_gt)
image_data_no_gt_df = pd.DataFrame(image_data_no_gt)
image_data_test_df = pd.DataFrame(image_data_test)
all_image_data_df = pd.DataFrame(all_image_data)
image_data_gt_df.set_index("filepath", inplace=True)
image_data_no_gt_df.set_index("filepath", inplace=True)
image_data_test_df.set_index("filepath", inplace=True)
all_image_data_df.set_index("filepath", inplace=True)

image_data_gt_df.to_csv("output/gt.csv")
image_data_no_gt_df.to_csv("output/no_gt.csv")
image_data_test_df.to_csv("output/test.csv")
all_image_data_df.to_csv("output/all.csv")

with open(gt_cache_path, 'wb') as fid:
    pickle.dump(image_data_gt, fid, 2)
with open(no_gt_cache_path, 'wb') as fid:
    pickle.dump(image_data_no_gt, fid, 2)
with open(test_cache_path, 'wb') as fid:
    pickle.dump(image_data_test, fid, 2)

In [171]:
image_data_gt_df.head()

,bboxes,ignoreareas
filepath,,
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492-ss0-z33.jpg,"[[106, 342, 115, 351]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059-ss0-z57.jpg,"[[404, 333, 412, 341]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059-ss0-z68.jpg,"[[414, 273, 439, 298]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987-ss0-z187.jpg,"[[424, 348, 430, 355]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565-ss0-z41.jpg,"[[453, 225, 462, 233]]",[]


In [172]:
image_data_no_gt_df.head()

,bboxes,ignoreareas
filepath,,
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260-ss0-z0.jpg,[],[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260-ss0-z1.jpg,[],[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260-ss0-z2.jpg,[],[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260-ss0-z3.jpg,[],[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260-ss0-z4.jpg,[],[]


In [173]:
image_data_test_df.head()

,bboxes,ignoreareas
filepath,,
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.102681962408431413578140925249-ss9-z121.jpg,"[[342, 267, 366, 291]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437-ss9-z57.jpg,"[[227, 302, 235, 309]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437-ss9-z64.jpg,"[[87, 225, 92, 231]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437-ss9-z69.jpg,"[[387, 281, 399, 292]]",[]
data/images/1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295249119452142211437-ss9-z82.jpg,"[[127, 230, 134, 237]]",[]
